# ReLU-Based Robust Classifier via the Convex Outer Adversarial Polytope - SVHN Dataset

In this notebook, two state-of-the-art CNNs of 4 convolutional layers prior to the fully-connected part are trained on the SVHN normally and robustly using the the "Convex Outer Adversarial Polytope" described in [Provable Defenses against Adversarial Examples via the Convex Outer Adversarial Polytope](https://arxiv.org/pdf/1711.00851.pdf) by Wong and Kolter. This notebook also contains the code to convert neural networks from PyTorch to Keras and vice versa for completeness with the rest of the notebooks of this project.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import seaborn
import matplotlib.patches as patches
from scipy.spatial import HalfspaceIntersection
%matplotlib inline
seaborn.set(font_scale=2)
seaborn.set_style("white")
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import argparse
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten ,Input
from keras.layers import Conv2D, MaxPooling2D, Reshape, Add
from keras.metrics import categorical_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from tensorflow.python.keras import backend as K
from keras import models
from keras import layers
from keras.preprocessing.image import array_to_img, img_to_array

In [ ]:
!pip install convex_adversarial

# Dataset

In [ ]:
from scipy.io import loadmat
import shutil
from sklearn.model_selection import train_test_split
import keras

x_train = loadmat('/content/drive/My Drive/Colab Notebooks/SVHN/train_32x32.mat')
x_acc_test = loadmat('/content/drive/My Drive/Colab Notebooks/SVHN/test_32x32.mat')

y_train = x_train['y']
x_train = x_train['X']
y_acc_test = x_acc_test['y']
x_acc_test = x_acc_test['X']

x_train = np.rollaxis(x_train, 3)
x_acc_test = np.rollaxis(x_acc_test, 3)

y_train = y_train[:,0]
y_acc_test = y_acc_test[:,0]

y_train[y_train==10] = 0
y_acc_test[y_acc_test==10] = 0

#Split between Training and Validation datasets (x_train = train & x_test = validation)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state = 100)

# Create labels and one-hot-encoding
y_labels_train = y_train
y_labels_test = y_test
y_labels_acc_test = y_acc_test
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
y_acc_test = keras.utils.to_categorical(y_acc_test, 10)

In [ ]:
#Preprocess data for CNN
x_train_cnn = x_train.reshape(-1, 32, 32, 3)
x_train_cnn = x_train_cnn.astype('float32')
x_train_cnn /= 255.

x_test_cnn = x_test.reshape(-1, 32, 32, 3)
x_test_cnn = x_test_cnn.astype('float32')
x_test_cnn /= 255.

x_acc_test_cnn = x_acc_test.reshape(-1, 32, 32, 3)
x_acc_test_cnn = x_acc_test_cnn.astype('float32')
x_acc_test_cnn /= 255.

# Save
np.save('x_train_cnn', x_train_cnn)
np.save('x_acc_test_cnn', x_acc_test_cnn)
np.save('x_test_cnn', x_test_cnn)

shutil.move("/content/x_train_cnn.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_train_cnn.npy")
shutil.move("/content/x_acc_test_cnn.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_acc_test_cnn.npy")
shutil.move("/content/x_test_cnn.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_test_cnn.npy")

# Save the labels
np.save('y_labels_train', y_labels_train)
np.save('y_labels_test', y_labels_test)
np.save('y_labels_acc_test', y_labels_acc_test)
np.save('y_train', y_train)
np.save('y_test', y_test)
np.save('y_acc_test', y_acc_test)

shutil.move("/content/y_labels_train.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_train.npy")
shutil.move("/content/y_labels_test.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_test.npy")
shutil.move("/content/y_labels_acc_test.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_acc_test.npy")
shutil.move("/content/y_train.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_train.npy")
shutil.move("/content/y_test.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_test.npy")
shutil.move("/content/y_acc_test.npy", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_acc_test.npy")

# From PyTorch to Keras

In [ ]:
import numpy as np
import torch
import keras

def pyt_to_keras(pytorch_model, keras_model):
  pyt_state_dict = pytorch_model.state_dict()
  for idx, layer in enumerate(keras_model.layers):
    if type(layer).__name__.endswith('Conv2D'):
      # Keras 2D Convolutional layer: height * width * input channels * output channels
      # PyTorch 2D Convolutional layer: output channels * input channels * height * width
      name = layer.name
      weights = np.transpose(pyt_state_dict[name + '.weight'].numpy(), (2, 3, 1, 0))
      bias = pyt_state_dict[name + '.bias'].numpy()
      keras_model.layers[idx].set_weights([weights, bias])
    elif type(layer).__name__.endswith('Dense'):
      # Keras Linear Layer: input neurons * output neurons
      # PyTorch Linear Layer: output neurons * input neurons
      name = layer.name
      weights = np.transpose(pyt_state_dict[name + '.weight'].numpy(), (1, 0))
      bias = pyt_state_dict[name + '.bias'].numpy()
      keras_model.layers[idx].set_weights([weights, bias])
  return keras_model

In [ ]:
!pip install torchsummary

# Train Baseline

In [ ]:
def CNN_model():
  model = nn.Sequential(
        nn.Conv2d(3, 16,  4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 16, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 32, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(32*2*2,100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )
  return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = CNN_model()
model.cuda()
from torchsummary import summary

summary(model, (3, 32, 32))
print (model)

In [ ]:
!pip install setproctitle

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import setproctitle
import argparse
import problems as pblm
from trainer import *
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

prefix = "/content/SVHN_robust/"

train_log = open(prefix + "train_baseline.log", "w")
test_log = open(prefix + "test_baseline.log", "w")

# Restore the data
x_train_cnn_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_train_cnn.npy')

x_train_cnn=[]
for i in range(58605):
  test = x_train_cnn_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_train_cnn.append(np.array(test_new))
x_train_cnn = np.array(x_train_cnn) 

x_test_cnn_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_test_cnn.npy')
x_test_cnn=[]
for i in range(14652):
  test = x_test_cnn_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_test_cnn.append(np.array(test_new))
x_test_cnn = np.array(x_test_cnn)

y_train=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_train.npy')
y_test=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_test.npy')

tensor_x = torch.tensor(x_train_cnn, dtype=torch.float) # transform to torch tensor
tensor_y = torch.tensor(y_train, dtype=torch.long)
dataset_train = TensorDataset(tensor_x, tensor_y) # create your datset
train_loader = DataLoader(dataset_train,batch_size=20, shuffle=False) # create your dataloader

tensor_x_test = torch.tensor(x_test_cnn) # transform to torch tensor
tensor_y_test = torch.tensor(y_test, dtype=torch.long)
dataset_test = TensorDataset(tensor_x_test, tensor_y_test) # create your datset
test_loader = DataLoader(dataset_test, batch_size=20, shuffle=False) # create your dataloader

train_loader, test_loader = pblm.svhn_loaders(20)

torch.manual_seed(0)
torch.cuda.manual_seed(0)

model = CNN_model().cuda()

opt = optim.Adam(model.parameters(), lr=0.001)

epsilon = 0.01
batch_size=20
epochs=20

starting_epsilon=None
baseline=True
alpha_grad=True
scatter_grad=True

verbose = 0

for epoch in range(epochs):
  batch_time = AverageMeter()
  data_time = AverageMeter()
  losses = AverageMeter()
  errors = AverageMeter()

  model.train()

  end = time.time()
  for i, (X,y) in enumerate(train_loader):
      X,y = X.cuda(), y.cuda()
      data_time.update(time.time() - end)

      out = model(Variable(X))
      ce = nn.CrossEntropyLoss()(out, Variable(y))
      err = (out.data.max(1)[1] != y).float().sum()  / X.size(0)

      opt.zero_grad()
      ce.backward()
      opt.step()

      batch_time.update(time.time()-end)
      end = time.time()
      losses.update(ce.item(), X.size(0))
      errors.update(err, X.size(0))

      print(epoch, i, ce.item(), err, file=train_log)
      if verbose and i % verbose == 0:
          print('Epoch: [{0}][{1}/{2}]\t'
                'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Error {errors.val:.3f} ({errors.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, errors=errors))
      train_log.flush()

  batch_time = AverageMeter()
  losses = AverageMeter()
  errors = AverageMeter()

  model.eval()

  end = time.time()
  for i, (X,y) in enumerate(test_loader):
      X,y = X.cuda(), y.cuda()
      out = model(Variable(X))
      ce = nn.CrossEntropyLoss()(out, Variable(y))
      err = (out.data.max(1)[1] != y).float().sum()  / X.size(0)

      # print to logfile
      print(epoch, i, ce.item(), err, file=test_log)

      # measure accuracy and record loss
      losses.update(ce.item(), X.size(0))
      errors.update(err, X.size(0))

      # measure elapsed time
      batch_time.update(time.time() - end)
      end = time.time()

      if verbose and i % verbose == 0:
          print('Test: [{0}/{1}]\t'
                'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Error {error.val:.3f} ({error.avg:.3f})'.format(
                    i, len(test_loader), batch_time=batch_time, loss=losses,
                    error=errors))
      test_log.flush()

  print(' * Error {error.avg:.4f}\t'
        'Loss {loss.avg:.4f}'
        .format(error=errors, loss=losses))
  torch.save(model.state_dict(), prefix +str(epoch)+ "_Baseline_model.pth")

#torch.save(model.state_dict(), prefix + "F_Baseline_model.pth")

# Test Baseline

In [ ]:
model = CNN_model().cpu()
model.load_state_dict(torch.load('/content/SVHN_robust/0_Baseline_model.pth'))
model.eval()

x_acc_test_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_acc_test_cnn.npy')
x_acc_test=[]
for i in range(26032):
  test = x_acc_test_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_acc_test.append(np.array(test_new))
x_acc_test = np.array(x_acc_test)

y_test_acc=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_acc_test.npy')

tensor_x_test_acc = torch.tensor(x_acc_test) # transform to torch tensor
tensor_y_test_acc = torch.tensor(y_test_acc, dtype=torch.long)
dataset_test_acc = TensorDataset(tensor_x_test_acc, tensor_y_test_acc) # create your datset
test_loader_acc = DataLoader(dataset_test_acc, batch_size=20, shuffle=False) # create your dataloader

correct = 0
total = 0
predictions = []
with torch.no_grad():
    for data in test_loader_acc:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 26032 test images: %f %%' % (
    100 * correct / total))

In [ ]:
print (model)

# Baseline From PyTorch to Keras

In [ ]:
def get_CNN_model():
  #CNN network for classification
  svhn_model = models.Sequential(name='CNN')
  svhn_model.add(layers.Conv2D(16, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', input_shape=(3, 32, 32),  name='0'))
  svhn_model.add(layers.Conv2D(16, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', name='2'))
  svhn_model.add(layers.Conv2D(32, 4,  data_format='channels_first',activation='relu', strides = 2, padding='same', name='4'))
  svhn_model.add(layers.Conv2D(32, 4,  data_format='channels_first',activation='relu', strides = 2, padding='same', name='6'))
  svhn_model.add(Flatten(name='8'))
  svhn_model.add(layers.Dense(100, activation='relu', name='9'))
  svhn_model.add(layers.Dense(10, name='11'))
  return svhn_model

keras_model = get_CNN_model()
keras_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[categorical_accuracy])
keras_model.summary()

In [ ]:
keras_model = get_CNN_model()
keras_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[categorical_accuracy])

model = CNN_model()
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/0_Baseline_model.pth'))
model.eval()

keras_model = pyt_to_keras(model, keras_model)
keras_model.save('/content/SVHN_robust/Baseline_Keras_78.h5')
print ('Keras Model Saved')

In [ ]:
# Evaluate CNN
x_acc_test_cnn = np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_acc_test_cnn.npy')
y_acc_test=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_acc_test.npy')

keras_model = get_CNN_model()
keras_model = keras.models.load_model('/content/SVHN_robust/Baseline_Keras_78.h5')
score, acc = keras_model.evaluate(x_acc_test, y_acc_test , batch_size=128)
print ("Test Accuracy: %.5f" % acc)

In [ ]:
data = torch.rand(6,3,32,32)
data_keras = data.numpy()
data_pytorch = Variable(data, requires_grad=False)

# Do a forward pass in both frameworks
keras_pred = keras_model.predict(data_keras)
pytorch_pred = model(data_pytorch).data.numpy()
assert keras_pred.shape == pytorch_pred.shape

plt.axis('Off')
plt.imshow(keras_pred)
plt.show()
plt.axis('Off')
plt.imshow(pytorch_pred)
plt.show()

# Train Robustly

In [ ]:
def CNN_model():
  model = nn.Sequential(
        nn.Conv2d(3, 16,  4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(32*8*8,100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )
  return model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import setproctitle
import argparse
import problems as pblm
from trainer import *
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

import shutil

prefix = "/content/SVHN_robust/"

train_log = open(prefix + "train_robust.log", "w")
test_log = open(prefix + "test_robust.log", "w")

# Restore the data
x_train_cnn_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_train_cnn.npy')
x_train_cnn=[]
for i in range(58605):
  test = x_train_cnn_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_train_cnn.append(np.array(test_new))
x_train_cnn = np.array(x_train_cnn) 

x_test_cnn_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_test_cnn.npy')
x_test_cnn=[]
for i in range(14652):
  test = x_test_cnn_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_test_cnn.append(np.array(test_new))
x_test_cnn = np.array(x_test_cnn)

y_train=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_train.npy')
y_test=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_test.npy')

tensor_x = torch.tensor(x_train_cnn, dtype=torch.float) # transform to torch tensor
tensor_y = torch.tensor(y_train, dtype=torch.long)
dataset_train = TensorDataset(tensor_x, tensor_y) # create your datset
train_loader = DataLoader(dataset_train,batch_size=20, shuffle=False) # create your dataloader

tensor_x_test = torch.tensor(x_test_cnn) # transform to torch tensor
tensor_y_test = torch.tensor(y_test, dtype=torch.long)
dataset_test = TensorDataset(tensor_x_test, tensor_y_test) # create your datset
test_loader = DataLoader(dataset_test, batch_size=20, shuffle=False) # create your dataloader

train_loader, test_loader = pblm.svhn_loaders(20)

torch.manual_seed(0)
torch.cuda.manual_seed(0)

model = CNN_model().cuda()
#model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/1_Robust_model.pth'))

opt = optim.Adam(model.parameters(), lr=0.001)

epsilon_final = 0.01
batch_size=20
epochs=15

alpha_grad=False
scatter_grad=False

verbose = 0
real_time=False
parallel=True

starting_epsilon = 0.001

for epoch in range (epochs):
  batch_time = AverageMeter()
  data_time = AverageMeter()
  losses = AverageMeter()
  errors = AverageMeter()
  robust_losses = AverageMeter()
  robust_errors = AverageMeter()

  model.train()

  end = time.time()
  for i, (X,y) in enumerate(train_loader):
      X,y = X.cuda(), y.cuda().long()
      if y.dim() == 2:
          y = y.squeeze(1)
      data_time.update(time.time() - end)

      with torch.no_grad():
          out = model(Variable(X))
          ce = nn.CrossEntropyLoss()(out, Variable(y))
          err = (out.max(1)[1] != y).float().sum()  / X.size(0)

      if epoch <= epochs//2 and starting_epsilon is not None: 
        epsilon = starting_epsilon + (epoch/(epochs//2))*(epsilon_final - starting_epsilon)
      else:
        epsilon = epsilon_final

      robust_ce, robust_err = robust_loss(model, epsilon, Variable(X), Variable(y))
      opt.zero_grad()

      robust_ce.backward(torch.ones_like(robust_ce))

      opt.step()

      # measure accuracy and record loss
      losses.update(ce.item(), X.size(0))
      errors.update(err.item(), X.size(0))
      robust_losses.update(robust_ce.detach().item(), X.size(0))
      robust_errors.update(robust_err, X.size(0))

      # measure elapsed time
      batch_time.update(time.time()-end)
      end = time.time()

      print(epoch, i, robust_ce.detach().item(),
              robust_err, ce.item(), err.item(), file=train_log)

      if verbose and (i % verbose == 0 or real_time):
          endline = '\n' if i % verbose == 0 else '\r'
          print('Epoch: [{0}][{1}/{2}]\t'
                'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                'Robust loss {rloss.val:.4f} ({rloss.avg:.4f})\t'
                'Robust error {rerrors.val:.3f} ({rerrors.avg:.3f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Error {errors.val:.3f} ({errors.avg:.3f})'
                .format(
                  epoch, i, len(train_loader), 
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses,
                  errors=errors,
                  rloss = robust_losses,
                  rerrors = robust_errors)
                , end=endline)
      train_log.flush()

      del X, y, robust_ce, out, ce, err, robust_err
        
  torch.cuda.empty_cache()

  batch_time = AverageMeter()
  losses = AverageMeter()
  errors = AverageMeter()
  robust_losses = AverageMeter()
  robust_errors = AverageMeter()

  model.eval()

  end = time.time()

  torch.set_grad_enabled(False)
  for i, (X,y) in enumerate(test_loader):
      X,y = X.cuda(), y.cuda().long()
      if y.dim() == 2: 
          y = y.squeeze(1)

      robust_ce, robust_err = robust_loss(model, epsilon_final, X, y)

      out = model(Variable(X))
      ce = nn.CrossEntropyLoss()(out, Variable(y))
      err = (out.max(1)[1] != y).float().sum()  / X.size(0)

      # measure accuracy and record loss
      losses.update(ce.item(), X.size(0))
      errors.update(err, X.size(0))
      robust_losses.update(robust_ce.item(), X.size(0))
      robust_errors.update(robust_err, X.size(0))

      # measure elapsed time
      batch_time.update(time.time()-end)
      end = time.time()

      print(epoch, i, robust_ce.item(), robust_err, ce.item(), err.item(),
          file=test_log)
      if verbose: 
          endline = '\n' if i % verbose == 0 else '\r'
          print('Test: [{0}/{1}]\t'
                'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                'Robust loss {rloss.val:.3f} ({rloss.avg:.3f})\t'
                'Robust error {rerrors.val:.3f} ({rerrors.avg:.3f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Error {error.val:.3f} ({error.avg:.3f})'.format(
                    i, len(test_loader), batch_time=batch_time, 
                    loss=losses, error=errors, rloss = robust_losses, 
                    rerrors = robust_errors), end=endline)
      test_log.flush()

      del X, y, robust_ce, out, ce

  torch.set_grad_enabled(True)
  torch.cuda.empty_cache()
  print(str(epoch))
  print(' * Robust error {rerror.avg:.3f}\t'
        'Error {error.avg:.3f}\t'
        'Robust loss {rloss.avg:.3f}\t'
        'Loss {loss.avg:.4f}'
        .format(rerror=robust_errors, error=errors, rloss=robust_losses, loss=losses))
  print (' ')

  torch.save(model.state_dict(), prefix +str(epoch)+ "_!Robust_model.pth")
  shutil.move(prefix +str(epoch)+ "_!Robust_model.pth", "/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/"+str(epoch)+ "_!Robust_model.pth")


# Test Robustly

In [ ]:
model = CNN_model()
model.load_state_dict(torch.load('/content/3_Robust_model.pth'))
model.eval()

x_acc_test_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_acc_test_cnn.npy')
x_acc_test=[]
for i in range(26032):
  test = x_acc_test_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_acc_test.append(np.array(test_new))
x_acc_test = np.array(x_acc_test)

y_test_acc=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_acc_test.npy')

tensor_x_test_acc = torch.tensor(x_acc_test) # transform to torch tensor
tensor_y_test_acc = torch.tensor(y_test_acc, dtype=torch.long)
dataset_test_acc = TensorDataset(tensor_x_test_acc, tensor_y_test_acc) # create your datset
test_loader_acc = DataLoader(dataset_test_acc, batch_size=20, shuffle=False) # create your dataloader

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader_acc:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %4f %%' % (
    100 * correct / total))

# Robust From PyTorch to Keras

In [ ]:
def get_CNN_model():
  #CNN network for classification
  svhn_model = models.Sequential(name='CNN')
  svhn_model.add(layers.Conv2D(16, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', input_shape=(3, 32, 32),  name='0'))
  svhn_model.add(layers.Conv2D(16, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', name='2'))
  svhn_model.add(layers.Conv2D(32, 4,  data_format='channels_first',activation='relu', strides = 2, padding='same', name='4'))
  svhn_model.add(layers.Conv2D(32, 4,  data_format='channels_first',activation='relu', strides = 2, padding='same', name='6'))
  svhn_model.add(Flatten(name='8'))
  svhn_model.add(layers.Dense(100, activation='relu', name='9'))
  svhn_model.add(layers.Dense(10, name='11'))
  return svhn_model

keras_model = get_CNN_model()
keras_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[categorical_accuracy])
keras_model.summary()

In [ ]:
def get_SCNN_model_layer2(channels_l1, channels_l2):
  #CNN network for classification
  svhn_model = models.Sequential(name='SCNN')
  svhn_model.add(layers.Conv2D(channels_l1, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', input_shape=(3, 32, 32),  name='0'))
  svhn_model.add(layers.Conv2D(channels_l2, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', name='2'))
  svhn_model.add(Flatten(name='4'))
  svhn_model.add(layers.Dense(100, activation='relu', name='5'))
  svhn_model.add(layers.Dense(10, name="7"))
  return svhn_model

In [ ]:
keras_model = get_SCNN_model_layer2(16, 32)
keras_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[categorical_accuracy])

model = CNN_model()
model.load_state_dict(torch.load('/content/3_Robust_model.pth'))
model.eval()

keras_model = pyt_to_keras(model, keras_model)
keras_model.save('/content/Robust_ReTrain_Keras.h5')
print ('Keras Model Saved')

In [ ]:
# Evaluate CNN
x_acc_test_cnn = np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_acc_test_cnn.npy')
y_acc_test=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_acc_test.npy')

keras_model = get_SCNN_model_layer2(16, 32)
keras_model = keras.models.load_model('/content/Robust_ReTrain_Keras.h5')
score, acc = keras_model.evaluate(x_acc_test, y_acc_test , batch_size=128)
print ("Test Accuracy: %.5f" % acc)

In [ ]:
data = torch.rand(6,3,32,32)
data_keras = data.numpy()
data_pytorch = Variable(data, requires_grad=False)

# Do a forward pass in both frameworks
keras_pred = keras_model.predict(data_keras)
pytorch_pred = model(data_pytorch).data.numpy()
assert keras_pred.shape == pytorch_pred.shape

plt.axis('Off')
plt.imshow(keras_pred)
plt.show()
plt.axis('Off')
plt.imshow(pytorch_pred)
plt.show()

# Robust Distillated From Keras to PyTorch

In [ ]:
def get_SCNN_model_layer2(channels_l1, channels_l2):
  #CNN network for classification
  svhn_model = models.Sequential(name='SCNN')
  svhn_model.add(layers.Conv2D(channels_l1, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', input_shape=(3, 32, 32),  name='0'))
  svhn_model.add(layers.Conv2D(channels_l2, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', name='2'))
  svhn_model.add(Flatten(name='8'))
  svhn_model.add(layers.Dense(100, activation='relu', name='9'))
  svhn_model.add(layers.Dense(10, name="logit"))
  return svhn_model

def get_CNN_model():
  #CNN network for classification
  svhn_model = models.Sequential(name='CNN')
  svhn_model.add(layers.Conv2D(16, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', input_shape=(3, 32, 32),  name='0'))
  svhn_model.add(layers.Conv2D(16, 4, data_format='channels_first', activation='relu', strides = 2, padding='same', name='2'))
  svhn_model.add(layers.Conv2D(32, 4,  data_format='channels_first',activation='relu', strides = 2, padding='same', name='4'))
  svhn_model.add(layers.Conv2D(32, 4,  data_format='channels_first',activation='relu', strides = 2, padding='same', name='6'))
  svhn_model.add(Flatten(name='8'))
  svhn_model.add(layers.Dense(100, activation='relu', name='9'))
  svhn_model.add(layers.Dense(10, name='11'))
  return svhn_model

keras_network = get_SCNN_model_layer2(16, 32)
keras_network = keras.models.load_model('/content/SCNN_MIMIC_SVHN_h16_32.h5')

for i, layer in enumerate(keras_network.layers):
    if i == 1:
      layer.name = str(0)
    elif i == 3:
      layer.name = str(4)
    elif i == 4:
      layer.name = str(5)
    elif i == 5:
      layer.name = str(7)

keras_network.summary()

In [ ]:
def CNN_model():
  model = nn.Sequential(
        nn.Conv2d(3, 16,  4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(32*8*8,100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )
  return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = CNN_model()
model.cuda()
from torchsummary import summary

summary(model, (3, 32, 32))
print (model)

In [ ]:
from nn_transfer import transfer, util

pytorch_network = CNN_model()
transfer.keras_to_pytorch(keras_network, pytorch_network)
torch.save(pytorch_network.state_dict(),"SCNN_MIMIC_SVHN_h16_32.pth")

In [ ]:
x_acc_test_old=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/x_acc_test_cnn.npy')
x_acc_test=[]
for i in range(26032):
  test = x_acc_test_old[i,:,:,:].T
  test_new = []
  for j in range(3):
    test_new.append(test[j].T)
  x_acc_test.append(np.array(test_new))
x_acc_test = np.array(x_acc_test)

y_test_acc=np.load('/content/drive/My Drive/Colab Notebooks/SVHN/robust_convex/y_labels_acc_test.npy')

tensor_x_test_acc = torch.tensor(x_acc_test) # transform to torch tensor
tensor_y_test_acc = torch.tensor(y_test_acc, dtype=torch.long)
dataset_test_acc = TensorDataset(tensor_x_test_acc, tensor_y_test_acc) # create your datset
test_loader_acc = DataLoader(dataset_test_acc, batch_size=20, shuffle=False) # create your dataloader

model = CNN_model()
model.load_state_dict(torch.load('/content/SCNN_MIMIC_SVHN_h16_32.pth'))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader_acc:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %4f %%' % (
    100 * correct / total))